# Apply

Die am allgemeinsten einsetzbare `GroupBy`-Methode ist `apply`. Sie teilt das zu bearbeitende Objekt  auf, ruft die übergebene Funktion auf jedem Teil auf und versucht dann, die Teile miteinander zu verketten.

Nehmen wir an, wir wollen die fünf größten `hit`-Werte nach Gruppen auswählen. Hierzu schreiben wir zunächst eine Funktion, die die Zeilen mit den größten Werten in einer bestimmten Spalte auswählt:

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame({'Title' : ['Jupyter Tutorial',
                              'Jupyter Tutorial',
                              'PyViz Tutorial',
                              None,
                              'Python Basics',
                              'Python Basics'],
                   '2021-12' : [30134,6073,4873,None,427,95],
                   '2022-01' : [33295,7716,3930,None,276,226],
                   '2022-02' : [19651,6547,2573,None,525,157]})

df

,Title,2021-12,2022-01,2022-02
0,Jupyter Tutorial,30134.0,33295.0,19651.0
1,Jupyter Tutorial,6073.0,7716.0,6547.0
2,PyViz Tutorial,4873.0,3930.0,2573.0
3,None,NaN,NaN,NaN
4,Python Basics,427.0,276.0,525.0
5,Python Basics,95.0,226.0,157.0


In [3]:
def top(df, n=5, column='2021-12'):
    return df.sort_values(by=column, ascending=False)[:n]

top(df, n=3)

,Title,2021-12,2022-01,2022-02
0,Jupyter Tutorial,30134.0,33295.0,19651.0
1,Jupyter Tutorial,6073.0,7716.0,6547.0
2,PyViz Tutorial,4873.0,3930.0,2573.0


Wenn wir nun z.B. nach Titeln gruppieren und `apply` mit dieser Funktion aufrufen, erhalten wir Folgendes:

In [4]:
grouped = df.groupby('Title')

grouped.apply(top)

Title  2021-12  2022-01  2022-02
Title                                                          
Jupyter Tutorial 0  Jupyter Tutorial  30134.0  33295.0  19651.0
                 1  Jupyter Tutorial   6073.0   7716.0   6547.0
PyViz Tutorial   2    PyViz Tutorial   4873.0   3930.0   2573.0
Python Basics    4     Python Basics    427.0    276.0    525.0
                 5     Python Basics     95.0    226.0    157.0

Was ist hier passiert? Die obere Funktion wird für jede Zeilengruppe des DataFrame aufgerufen, und dann werden die Ergebnisse mit [pandas.concat](https://pandas.pydata.org/docs/reference/api/pandas.concat.html) zusammengefügt, wobei die Teile mit den Gruppennamen gekennzeichnet werden. Das Ergebnis hat daher einen hierarchischen Index, dessen innere Ebene Indexwerte aus dem ursprünglichen DataFrame enthält.

Wenn ihr eine Funktion an `apply` übergebt, die andere Argumente oder Schlüsselwörter benötigt, könnt ihr diese nach der Funktion übergeben:

In [5]:
grouped.apply(top, n=1)

,,Title,2021-12,2022-01,2022-02
Title,,,,,
Jupyter Tutorial,0,Jupyter Tutorial,30134.0,33295.0,19651.0
PyViz Tutorial,2,PyViz Tutorial,4873.0,3930.0,2573.0
Python Basics,4,Python Basics,427.0,276.0,525.0


Wir haben nun die grundlegende Verwendungsweise von `apply` gesehen. Was innerhalb der übergebenen Funktion geschieht, ist sehr vielseitig und bleibt euch überlassen; sie muss nur ein pandas-Objekt oder einen Einzelwert zurückgeben. Im Folgend werden wir daher hauptsächlich Beispielen zeigen, die euch Anregungen geben können, wie ihr verschiedene Probleme mit `groupby` lösen könnt.

Zunächst vergegenwärtigen wir uns nochmal an `describe`, aufgerufen über dem `GroupBy`-Objekt:

In [6]:
result = grouped.describe()

result

2021-12                                                    \
                   count     mean           std     min       25%      50%   
Title                                                                        
Jupyter Tutorial     2.0  18103.5  17013.696262  6073.0  12088.25  18103.5   
PyViz Tutorial       1.0   4873.0           NaN  4873.0   4873.00   4873.0   
Python Basics        2.0    261.0    234.759451    95.0    178.00    261.0   

                                    2022-01           ...                     \
                       75%      max   count     mean  ...       75%      max   
Title                                                 ...                      
Jupyter Tutorial  24118.75  30134.0     2.0  20505.5  ...  26900.25  33295.0   
PyViz Tutorial     4873.00   4873.0     1.0   3930.0  ...   3930.00   3930.0   
Python Basics       344.00    427.0     2.0    251.0  ...    263.50    276.0   

                 2022-02                                                 \
                   count     mean          std     min     25%      50%   
Title                                                                     
Jupyter Tutorial     2.0  13099.0  9265.927261  6547.0  9823.0  13099.0   
PyViz Tutorial       1.0   2573.0          NaN  2573.0  2573.0   2573.0   
Python Basics        2.0    341.0   260.215295   157.0   249.0    341.0   

                                    
                      75%      max  
Title                               
Jupyter Tutorial  16375.0  19651.0  
PyViz Tutorial     2573.0   2573.0  
Python Basics       433.0    525.0  

[3 rows x 24 columns]

Wenn ihr innerhalb von `GroupBy` eine Methode wie `describe` aufruft, ist dies eigentlich nur eine Abkürzung für:

In [7]:
f = lambda x: x.describe()
grouped.apply(f)

2021-12       2022-01       2022-02
Title                                                           
Jupyter Tutorial count      2.000000      2.000000      2.000000
                 mean   18103.500000  20505.500000  13099.000000
                 std    17013.696262  18087.084356   9265.927261
                 min     6073.000000   7716.000000   6547.000000
                 25%    12088.250000  14110.750000   9823.000000
                 50%    18103.500000  20505.500000  13099.000000
                 75%    24118.750000  26900.250000  16375.000000
                 max    30134.000000  33295.000000  19651.000000
PyViz Tutorial   count      1.000000      1.000000      1.000000
                 mean    4873.000000   3930.000000   2573.000000
                 std             NaN           NaN           NaN
                 min     4873.000000   3930.000000   2573.000000
                 25%     4873.000000   3930.000000   2573.000000
                 50%     4873.000000   3930.000000   2573.000000
                 75%     4873.000000   3930.000000   2573.000000
                 max     4873.000000   3930.000000   2573.000000
Python Basics    count      2.000000      2.000000      2.000000
                 mean     261.000000    251.000000    341.000000
                 std      234.759451     35.355339    260.215295
                 min       95.000000    226.000000    157.000000
                 25%      178.000000    238.500000    249.000000
                 50%      261.000000    251.000000    341.000000
                 75%      344.000000    263.500000    433.000000
                 max      427.000000    276.000000    525.000000

## Unterdrückung der Gruppenschlüssel

In den vorangegangenen Beispielen habr ihr gesehen, dass das resultierende Objekt einen hierarchischen Index hat, der aus den Gruppenschlüsseln zusammen mit den Indizes der einzelnen Teile des ursprünglichen Objekts gebildet wird. Ihr können dies deaktivieren, indem ihr `group_keys=False` an `groupby` übergebt:

In [8]:
grouped = df.groupby('Title', group_keys=False)

grouped.apply(top)

,Title,2021-12,2022-01,2022-02
0,Jupyter Tutorial,30134.0,33295.0,19651.0
1,Jupyter Tutorial,6073.0,7716.0,6547.0
2,PyViz Tutorial,4873.0,3930.0,2573.0
4,Python Basics,427.0,276.0,525.0
5,Python Basics,95.0,226.0,157.0


## Quantil- und Bucket-Analyse

Wie bereits in  [Diskretisierung und Gruppierung](discretisation.ipynb) beschrieben, verfügt pandas über einige Werkzeuge, insbesondere `cut` und `qcut`, um Daten in Buckets mit Bins eurer Wahl oder nach Stichprobenquantilen aufzuteilen. Kombiniert man diese Funktionen mit `groupby`, kann man bequem eine Bucket- oder Quantilanalyse für einen Datensatz durchführen. Betrachtet einen einfachen Zufallsdatensatz und eine gleich lange Bucket-Kategorisierung mit `cut`:

In [9]:
df = pd.DataFrame({'data1': np.random.randn(1000),
                   'data2': np.random.randn(1000)})

quartiles = pd.cut(df.data1, 4)

quartiles[:10]

0      (1.283, 2.763]
1     (-0.197, 1.283]
2    (-1.678, -0.197]
3    (-1.678, -0.197]
4    (-1.678, -0.197]
5     (-0.197, 1.283]
6    (-1.678, -0.197]
7     (-0.197, 1.283]
8     (-0.197, 1.283]
9    (-1.678, -0.197]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-3.164, -1.678] < (-1.678, -0.197] < (-0.197, 1.283] < (1.283, 2.763]]

Das von `cut` zurückgegebene `Categorical`-Objekt kann direkt an `groupby` übergeben werden. Wir könnten also eine Reihe von Gruppenstatistiken für die Quartile wie folgt berechnen:

In [10]:
def stats(group):
    return pd.DataFrame(
        {'min': group.min(), 'max': group.max(),
         'count': group.count(), 'mean': group.mean()}
    )

grouped = df.groupby(quartiles)

grouped.apply(stats)

min       max  count      mean
data1                                                      
(-3.164, -1.678] data1 -3.158082 -1.722431     54 -2.078976
                 data2 -2.832506  1.956335     54 -0.009314
(-1.678, -0.197] data1 -1.673850 -0.197968    391 -0.749178
                 data2 -2.684316  3.049758    391  0.016235
(-0.197, 1.283]  data1 -0.189442  1.278557    453  0.456867
                 data2 -2.877565  2.750697    453 -0.016397
(1.283, 2.763]   data1  1.284391  2.763085    102  1.742284
                 data2 -2.130623  2.824928    102 -0.084833

Dies waren Buckets gleicher Länge; um Buckets gleicher Größe auf der Grundlage von Stichprobenquantilen zu berechnen, können wir `qcut` verwenden. Ich übergebe `labels=False`, um nur Quantilzahlen zu erhalten:

In [11]:
quartiles_samp = pd.qcut(df.data1, 4, labels=False)

grouped = df.groupby(quartiles_samp)

grouped.apply(stats)

min       max  count      mean
data1                                           
0     data1 -3.158082 -0.687638    250 -1.293436
      data2 -2.832506  2.453921    250  0.069764
1     data1 -0.685401 -0.008461    250 -0.349850
      data2 -2.859195  3.049758    250 -0.019565
2     data1 -0.008248  0.645475    250  0.298758
      data2 -2.877565  2.334934    250 -0.049703
3     data1  0.651807  2.763085    250  1.262449
      data2 -2.301336  2.824928    250 -0.041439

## Daten mit gruppenspezifischen Werten auffüllen

Wenn ihr fehlende Daten bereinigt, werdet ihr in einigen Fällen Datenbeobachtungen mit `dropna` ersetzen, aber in anderen Fällen möchtet ihr vielleicht die Nullwerte (`NA`) mit einem festen Wert oder einem aus den Daten abgeleiteten Wert auffüllen. `fillna` ist das richtige Werkzeug dafür; hier fülle ich zum Beispiel die Nullwerte mit dem Mittelwert auf:

In [12]:
s = pd.Series(np.random.randn(8))
s[::3] = np.nan

s

0         NaN
1    0.263561
2   -0.203659
3         NaN
4    1.102275
5   -0.696833
6         NaN
7    1.032439
dtype: float64

In [13]:
s.fillna(s.mean())

0    0.299557
1    0.263561
2   -0.203659
3    0.299557
4    1.102275
5   -0.696833
6    0.299557
7    1.032439
dtype: float64

Hier sind einige Beispieldaten zu meinen Tutorials, die in deutsch- und einglischsprachige Ausgaben unterteilt sind:

In [14]:
df = pd.DataFrame({'2021-12' : [30134,6073,4873,None,427,95],
                   '2022-01' : [33295,7716,3930,None,276,226],
                   '2022-02' : [19651,6547,2573,None,525,157]},
                  index=[['Jupyter Tutorial',
                          'Jupyter Tutorial',
                          'PyViz Tutorial',
                          'PyViz Tutorial',
                          'Python Basics',
                          'Python Basics'],
                         ['de', 'en', 'de', 'en', 'de', 'en']])
df.index.names = ['Title', 'Language']

df

2021-12  2022-01  2022-02
Title            Language                           
Jupyter Tutorial de        30134.0  33295.0  19651.0
                 en         6073.0   7716.0   6547.0
PyViz Tutorial   de         4873.0   3930.0   2573.0
                 en            NaN      NaN      NaN
Python Basics    de          427.0    276.0    525.0
                 en           95.0    226.0    157.0

Angenommen, ihr möchtet, dass der Füllwert je nach Gruppe variiert. Diese Werte können vordefiniert werden, und da die Gruppen ein internes Namensattribut `name` haben, könnt ihr dieses mit `apply` verwenden:

In [15]:
fill_values = {'de': 10632, 'en': 3469}

fill_func = lambda g: g.fillna(fill_values[g.name])

df.groupby('Language').apply(fill_func)

2021-12  2022-01  2022-02
Title            Language                           
Jupyter Tutorial de        30134.0  33295.0  19651.0
                 en         6073.0   7716.0   6547.0
PyViz Tutorial   de         4873.0   3930.0   2573.0
                 en         3469.0   3469.0   3469.0
Python Basics    de          427.0    276.0    525.0
                 en           95.0    226.0    157.0

Ihr könnt auch die Daten gruppieren und `apply` mit einer Funktion zu verwenden, die `fillna` für jedes Datenpaket aufruft:

In [16]:
fill_mean = lambda g: g.fillna(g.mean())

df.groupby('Language').apply(fill_mean)

2021-12  2022-01  2022-02
Title            Language                           
Jupyter Tutorial de        30134.0  33295.0  19651.0
                 en         6073.0   7716.0   6547.0
PyViz Tutorial   de         4873.0   3930.0   2573.0
                 en         3084.0   3971.0   3352.0
Python Basics    de          427.0    276.0    525.0
                 en           95.0    226.0    157.0